In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mlfinlab as ml
from alpha_vantage.timeseries import TimeSeries

In [3]:
ts = TimeSeries(key = 'PTME3FLJAHB9CY', output_format = 'pandas')

In [4]:
data, meta_data = ts.get_daily_adjusted(symbol = 'MSFT', outputsize = 'full')

In [9]:
data.plot()
plt.show()

/Users/marcusahlgren/.local/share/virtualenvs/financial_ML-WJfqPqib/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  
